2018 / 1 / 30

Wayne Nixalo

# RetinaNet Finetuning -- Dev 2

So... apparently Keras doesn't have a CSV data generator, but Keras-Retinanet Does.

In [1]:
import keras
import keras.preprocessing.image
import tensorflow as tf

/home/wnixalo/miniconda3/envs/aersu/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/wnixalo/miniconda3/envs/aersu/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [4]:
import keras_retinanet.losses
import keras_retinanet.layers
from keras_retinanet.preprocessing.csv_generator import CSVGenerator

from keras_retinanet.utils.image import preprocess_image, resize_image
from keras_retinanet.models.resnet import custom_objects

# from keras_retinanet.callbacks import RedirectModel
# from keras_retinanet.models.resnet import ResNet50RetinaNet
# from keras_retinanet.utils.keras_version import check_keras_version

In [3]:
def get_session():
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    return tf.Session(config=config)

Right now I'm working along with the Boring Detector training script: [link](https://github.com/lexfridman/boring-detector/blob/master/train.py#L16:6)

In [5]:
# parameters
batch_size = 1
steps_per_epoch = 10000
epochs = 50
# steps_per_training_epoch = 1500
# training_epochs = 10

### create model

In [ ]:
# create "base" model (no NMS) <-- what is `NMS`? <<-- Non-Maximum Suppresion
image = keras.layers.Input((None, None), 3)

model = keras.models.load_model('data/retinanet-model/resnet50_coco_best_v1.2.2.h5',
                                    custom_objects=custom_objects)
# model = ResNet50RetinaNet(image, num_classes=num_classes, weights='imagenet', nms=False)

training_model = model

# append NMS for prediction only
classification   = model.outputs[1]
detections       = model.outputs[2]
boxes            = keras.layers.Lambda(lambda x: x[:, :, :4])(detections)
detections       = keras_retinanet.layers.NonMaximumSuppression(name='nms')([boxes, classification, detections])
prediction_model = keras.models.Model(inputs=model.inputs, outputs=model.outputs[:2] + [detections])

# compile model
training_model.compile(
    loss={
        'regression'    : keras_retinanet.losses.smooth_l1(),
        'classification': keras_retinanet.losses.focal()
    },
    optimizer=keras.optimizers.adam(lr=1e-5, clipnorm=0.001)
)

### create generator

In [6]:
# create image data generator objects
train_image_data_generator = keras.preprocessing.image.ImageDataGenerator(
    horizontal_flip=True,
)

train_generator = CSVGenerator(
    boring_annotations_path,
    boring_classes_path,
    train_image_data_generator,
    batch_size=batch_size
)

NameError: name 'boring_annotations_path' is not defined

```
Init signature: CSVGenerator(csv_data_file, csv_class_file, image_data_generator, base_dir=None, **kwargs)
```

I have my model my infrastrcutre my everything else.. all I needed was to know a CSV datagenerator *existed* (I can check out how it was implemented later). So I can stop this follow-along here, fire up a version 3 of this notebook and crack at making this work on my own.